# IR Caplets Volatility Layout

In this Notebook, we explore different surface layouts for IR caplets volatility surface by varying key configuration layout. This process demonstrates the flexibility of **IR Caplets Volatility Surfaces** within the **LSEG Financial Analytics SDK**. 

We will explore the following surface layout and parameters options:

1. **Format:** `Matrix` - Different data structure representations
2. **Format:** `List` - Different data structure representations
3. **Input Volatility Type:** `NormalVolatility` (Bp) (default) vs `LogNormalVolatility` (%) - Different volatility conventions

## Imports

Import the following necessary modules:

- `ircaplet_volatility` - for Ir Caplet Volatility surface construction

This notebook uses external libraries **pandas, numpy, Ipython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.



In [1]:
from lseg_analytics.pricing.market_data import ircaplet_volatility as cv

import pandas as pd
import numpy as np
import json
import datetime as dt
from IPython.display import display

## 1. Caplets Volatility Surface - Format: `Matrix`

### Data Preparation

In [2]:
print("Step 1: Creating Surface Definition...")

currency = "USD"
index_name = "SOFR"

# Create surface definition object
surface_definition = cv.CapletsStrippingDefinition(
        instrument_code = currency,
        index_name = index_name,
        reference_caplet_tenor = "ON"
        )
print(f"   ✓ Instrument: {surface_definition.instrument_code}")

print("Step 2: Configuring Surface Parameters...") 
surface_parameters = cv.CapletsStrippingSurfaceParameters(
        calculation_date = dt.datetime.strptime("2025-01-18", "%Y-%m-%d"),
        x_axis = cv.XAxisEnum.STRIKE,                                    # Options: DATE, DELTA, EXPIRY, MONEYNESS, STRIKE, TENOR
        y_axis = cv.YAxisEnum.TENOR                                      # Options: same as X-axis
    )
print(f"   ✓ Surface Parameters: {surface_parameters}")

print("Step 3: Create request item...")
# Create the main request object with basic configuration
request_item = cv.CapletsStrippingSurfaceRequestItem(
        surface_tag = f"{currency}_CAPLET_VOLSURFACE",
        underlying_definition = surface_definition,
        surface_parameters = surface_parameters,
        underlying_type = cv.CurvesAndSurfacesUnderlyingTypeEnum.Cap,
        surface_layout = cv.SurfaceOutput(
            format = cv.FormatEnum.Matrix,  # Options: List, Matrix 
        )
    )
print(f"   ✓ Request Item: {json.dumps(request_item.as_dict(), indent=4)}")

Step 1: Creating Surface Definition...
   ✓ Instrument: USD
Step 2: Configuring Surface Parameters...
   ✓ Surface Parameters: {'calculationDate': '2025-01-18T00:00:00Z', 'xAxis': 'Strike', 'yAxis': 'Tenor'}
Step 3: Create request item...
   ✓ Request Item: {
    "surfaceTag": "USD_CAPLET_VOLSURFACE",
    "underlyingDefinition": {
        "instrumentCode": "USD",
        "indexName": "SOFR",
        "referenceCapletTenor": "ON"
    },
    "surfaceParameters": {
        "calculationDate": "2025-01-18T00:00:00Z",
        "xAxis": "Strike",
        "yAxis": "Tenor"
    },
    "underlyingType": "Cap",
    "surfaceLayout": {
        "format": "Matrix"
    }
}


### Request Execution

In [3]:
# Execute the calculation using the calculate function
try:
    response = cv.calculate(universe=[request_item])

    # Display response structure information
    surface_data = response['data'][0]
    if 'surface' in surface_data:
        print(f"   Calculation successful!")
        print(f"   Surface data points available: {len(surface_data['surface']) - 1} x {len(surface_data['surface'][0]) - 1}")
    else:
        print("   No surface data found in response")
    
except Exception as e:
    print(f"   Calculation failed: {str(e)}")
    raise

   Calculation successful!
   Surface data points available: 14 x 25


### Results Display

In [4]:
# Access surface matrix data from the response
surface_data = response['data'][0]['surface']

# Extract strikes (column headers) and tenors (row headers)
strikes = surface_data[0][1:]  # First row, excluding first element
tenors = [row[0] for row in surface_data[1:]]  # First column, excluding header row
volatility_matrix = np.array([[float(val) for val in row[1:]] for row in surface_data[1:]])

# Create DataFrame for easier manipulation and display
surface_df = pd.DataFrame(volatility_matrix, index=tenors, columns=strikes)

# Extract axis names for labeling plots
x_axis = surface_parameters.x_axis.name
y_axis = surface_parameters.y_axis.name

print("Surface DataFrame Info:") 
print(f"   Shape: {surface_df.shape} (rows × columns)") 
print(f"   x_axis: {x_axis}") 
print(f"   y_axis: {y_axis}") 

Surface DataFrame Info:
   Shape: (14, 25) (rows × columns)
   x_axis: STRIKE
   y_axis: TENOR


#### Caplets Normal Volatility Matrix (bp) - Format: `Matrix`

In [5]:
# Display all columns using context manager (temporary setting)
with pd.option_context('display.max_columns', None, 'display.width', None):
    display(surface_df)

,0.250000,0.500000,0.750000,1.000000,1.500000,2.000000,2.500000,3.000000,4.000000,4.009199,4.009324,4.010736,4.012121,4.012322,4.015504,4.019531,4.027613,4.028712,4.032534,4.045894,4.066390,4.112150,5.000000,6.000000,7.000000
3D,167.90,163.20,158.10,152.70,141.30,128.80,115.40,100.70,66.70,66.28,66.28,66.21,66.15,66.14,65.99,65.81,65.44,65.39,65.22,64.61,63.68,61.60,73.50,99.70,124.60
9M,167.90,163.20,158.10,152.70,141.30,128.80,115.40,100.70,66.70,66.28,66.28,66.21,66.15,66.14,65.99,65.81,65.44,65.39,65.22,64.61,63.68,61.60,73.50,99.70,124.60
1Y9M,129.84,128.59,127.14,125.80,122.90,120.46,118.77,118.43,126.21,126.03,126.03,126.00,125.97,125.97,125.91,125.83,125.67,125.65,125.64,125.61,125.56,125.45,123.35,125.76,137.49
2Y9M,128.29,126.77,125.17,123.18,119.19,115.33,111.42,108.28,103.14,102.61,102.60,102.52,102.44,102.44,102.48,102.53,102.64,102.65,102.70,102.87,103.13,103.71,115.00,131.57,146.63
3Y9M,115.66,114.75,113.92,113.06,111.40,110.05,109.17,109.54,115.14,115.73,115.74,115.75,115.75,115.75,115.76,115.78,115.81,115.81,115.83,115.87,115.95,116.12,119.39,126.85,137.92
4Y9M,110.54,109.81,108.49,107.74,106.09,104.38,103.52,103.32,104.40,104.34,104.34,104.35,104.36,104.36,104.39,104.43,104.50,104.51,104.54,104.66,104.84,105.24,113.00,123.93,135.39
5Y9M,103.72,103.11,102.55,101.68,100.35,99.71,99.57,99.93,104.90,104.96,104.96,104.97,104.97,104.98,104.99,105.01,105.05,105.05,105.07,105.13,105.23,105.45,109.66,118.33,128.49
6Y9M,102.51,101.79,101.34,100.49,99.72,98.07,98.00,98.45,100.28,100.89,100.90,101.00,101.09,101.10,101.13,101.16,101.22,101.23,101.26,101.37,101.53,101.89,108.88,117.77,128.09
7Y9M,97.91,97.52,96.82,96.22,94.65,95.20,94.78,95.67,100.55,100.28,100.28,100.24,100.20,100.19,100.10,100.13,100.17,100.18,100.20,100.28,100.39,100.66,105.76,114.21,124.19
8Y9M,96.40,95.30,95.02,94.55,94.04,92.58,93.21,92.89,96.23,96.89,96.90,97.00,97.10,97.11,97.34,97.63,97.69,97.70,97.72,97.81,97.95,98.25,104.18,112.39,122.21


## 2. Caplets Volatility Surface - Format: `List`

### Data Preparation
 Now we will Change the format from `Matrix` to `List`

In [6]:
# Change the format from Matrix to List
request_item.surface_layout.format = cv.FormatEnum.List
print(f"   ✓ Format changed to: {request_item.surface_layout.format.name}")

   ✓ Format changed to: LIST


### Request Execution

In [7]:
# Execute the calculation using the calculate function
try:
    response = cv.calculate(universe=[request_item])

    # Display response structure information
    surface_data = response['data'][0]
    if 'surface' in surface_data:
        print(f"   Calculation successful!")
        print(f"   Surface data points available: {len(surface_data['surface'])}")
    else:
        print("   No surface data found in response")
    
except Exception as e:
    print(f"   Calculation failed: {str(e)}")
    raise

   Calculation successful!
   Surface data points available: 350


### Results Display

In [8]:
# Access surface data from the response
surface_data = response['data'][0]['surface']
headers = response['data'][0]['headers']

print(f"Headers: {headers}")
print(f"Data format: List with {len(surface_data)} data points")

# Convert list format to DataFrame
surface_df = pd.DataFrame(surface_data, columns=headers)

# Display data types and structure
print("Surface DataFrame Info:")
print(f"   Shape: {surface_df.shape} (rows × columns)")
print(f"   Columns: {list(surface_df.columns)}")
print(f"   Data types: {surface_df.dtypes.to_dict()}")

Headers: ['Tenor', 'Normal Vol (bp)', 'StrikePercent']
Data format: List with 350 data points
Surface DataFrame Info:
   Shape: (350, 3) (rows × columns)
   Columns: ['Tenor', 'Normal Vol (bp)', 'StrikePercent']
   Data types: {'Tenor': dtype('O'), 'Normal Vol (bp)': dtype('O'), 'StrikePercent': dtype('O')}


#### Caplets Normal Volatility (bp) - Format: `List`

In [9]:
# Display first 10 rows for readability
# To display all data, use: display(surface_df)
display(surface_df.head(10))

,Tenor,Normal Vol (bp),StrikePercent
0,3D,167.90,0.250000
1,3D,163.20,0.500000
2,3D,158.10,0.750000
3,3D,152.70,1.000000
4,3D,141.30,1.500000
5,3D,128.80,2.000000
6,3D,115.40,2.500000
7,3D,100.70,3.000000
8,3D,66.70,4.000000
9,3D,66.28,4.009199


### 3. Caplets Volatility Surface - `LogNormalVolatility` (%)

### Data Preparation
 Here we will change input_volatility_type from `NormalVolatility` (bp) to `LogNormalVolatility` (%) and format from `List` to `Matrix`

In [10]:
# Change input volatility type using string value
surface_parameters.input_volatility_type = "LogNormalVolatility"
print(f"   ✓ Input Volatility Type changed to: {surface_parameters.input_volatility_type}")

# Change format back to Matrix for LogNormalVolatility demonstration
request_item.surface_layout.format = cv.FormatEnum.Matrix
print(f"   ✓ Format changed to: {request_item.surface_layout.format.name}")

   ✓ Input Volatility Type changed to: InputVolatilityTypeEnum.LOG_NORMAL_VOLATILITY
   ✓ Format changed to: MATRIX


### Request Execution

In [11]:
# Execute the calculation using the calculate function
try:
    response = cv.calculate(universe=[request_item])

    # Display response structure information
    surface_data = response['data'][0]
    if 'surface' in surface_data:
        print(f"   Calculation successful!")
        print(f"   Surface data points available: {len(surface_data['surface']) - 1} x {len(surface_data['surface'][0]) - 1}")
    else:
        print("   No surface data found in response")
    
except Exception as e:
    print(f"   Calculation failed: {str(e)}")
    raise

   Calculation successful!
   Surface data points available: 14 x 25


### Results Display

In [12]:
# Access surface matrix data from the response
surface_data = response['data'][0]['surface']

# Extract strikes (column headers) and tenors (row headers)
strikes = surface_data[0][1:]  # First row, excluding first element
tenors = [row[0] for row in surface_data[1:]]  # First column, excluding header row
volatility_matrix = np.array([[float(val) for val in row[1:]] for row in surface_data[1:]])

# Create DataFrame for easier manipulation and display
surface_df = pd.DataFrame(volatility_matrix, index=tenors, columns=strikes)

# Extract axis names for labeling plots
x_axis = surface_parameters.x_axis.name
y_axis = surface_parameters.y_axis.name

print("Surface DataFrame Info:") 
print(f"   Shape: {surface_df.shape} (rows × columns)") 
print(f"   x_axis: {x_axis}") 
print(f"   y_axis: {y_axis}") 


Surface DataFrame Info:
   Shape: (14, 25) (rows × columns)
   x_axis: STRIKE
   y_axis: TENOR


#### Caplets LogNormal Volatility Matrix (%)

In [13]:
# Display all columns using context manager (temporary setting)
with pd.option_context('display.max_columns', None, 'display.width', None):
    display(surface_df)

,0.250000,0.500000,0.750000,1.000000,1.500000,2.000000,2.500000,3.000000,4.000000,4.009199,4.009324,4.010736,4.012121,4.012322,4.015504,4.019531,4.027613,4.028712,4.032534,4.045894,4.066390,4.112150,5.000000,6.000000,7.000000
3D,131.47,100.18,83.23,71.67,55.84,44.77,36.11,28.86,16.49,16.37,16.36,16.35,16.33,16.32,16.28,16.23,16.12,16.10,16.05,15.87,15.60,14.98,16.26,20.15,23.23
9M,131.47,100.18,83.23,71.67,55.84,44.77,36.11,28.86,16.49,16.37,16.36,16.35,16.33,16.32,16.28,16.23,16.12,16.10,16.05,15.87,15.60,14.98,16.26,20.15,23.23
1Y9M,106.59,81.63,68.84,60.49,49.65,42.72,37.95,34.69,31.98,31.84,31.84,31.82,31.80,31.79,31.74,31.68,31.56,31.54,31.53,31.48,31.40,31.23,27.85,25.78,25.89
2Y9M,106.01,80.19,67.22,58.71,47.65,40.37,35.14,31.17,25.81,25.48,25.48,25.43,25.38,25.38,25.38,25.38,25.39,25.39,25.39,25.39,25.40,25.42,25.72,26.74,27.48
3Y9M,98.57,74.11,62.19,54.60,44.96,38.97,34.85,32.03,29.10,28.95,28.95,28.95,28.94,28.94,28.94,28.93,28.91,28.91,28.90,28.87,28.83,28.73,26.83,25.90,25.92
4Y9M,94.40,70.49,59.07,51.80,42.55,36.72,32.70,29.84,26.18,25.99,25.99,25.99,25.99,25.99,25.99,25.99,25.98,25.98,25.98,25.97,25.95,25.92,25.23,25.17,25.34
5Y9M,90.68,67.19,56.18,49.31,40.67,35.30,31.68,29.11,26.27,26.22,26.22,26.21,26.21,26.21,26.20,26.20,26.18,26.18,26.18,26.16,26.13,26.06,24.79,24.10,24.02
6Y9M,91.25,66.74,55.58,48.56,39.93,34.59,30.94,28.41,25.31,25.20,25.20,25.18,25.17,25.16,25.16,25.16,25.15,25.15,25.14,25.13,25.11,25.06,24.15,23.93,24.06
7Y9M,88.84,64.38,53.52,46.90,38.57,33.53,30.18,27.83,25.15,24.97,24.97,24.94,24.91,24.91,24.85,24.84,24.83,24.83,24.83,24.81,24.79,24.74,23.80,23.30,23.13
8Y9M,88.22,63.32,52.61,45.87,37.81,32.83,29.42,27.00,24.30,24.24,24.24,24.23,24.23,24.22,24.20,24.18,24.17,24.17,24.17,24.15,24.13,24.08,23.15,22.88,22.85
